# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Les images proviennent du jeu de données publiques *The Extended Yale Face Database B* disponible sur le site http://vision.ucsd.edu/~iskwak/ExtYaleDatabase/ExtYaleB.html.


## TD6 : Reconnaissance faciale avec l'analyse en composantes principales


Nous utiliserons des images récupérées de la base de données publique de Yale$^{(1)}$ que vous pouvez trouver sur Moodle. Nous avons choisi les 2 ensembles de telle sorte que l'ensemble de test contienne à la fois des images déjà vues et d'autres tout-à-fait nouvelles. L'objectif final est de mesurer la performance de notre algorithme en comptant le nombre d'exemples bien classifiés.

Plusieurs librairies que nous n'utilisons pas régulièrement dans le cours sont nécessaire. Exécutez la prochaine cellule de code pour installer ces librairies.

In [ ]:
import Pkg
Pkg.add(["Images","Netpbm","ImageMagick","Colors"])

In [ ]:
# Librairies standards du cours
using Statistics, LinearAlgebra, Gadfly, DataFrames

# Librairie pour le traitement des images
using Images, Netpbm, ImageMagick, Colors


# 1. Chargement des images d'entraînement
___

Les images d'entraînement sont contenues dans le dossier *Train* du jeu de données que vous pouvez récupérer sur Moodle. L'ensemble d'entraînement est constitué de 784 images provenant de 28 personnes différente, soit de 28 images par personne.

Les images sont des visages déjà correctement alignés, ce qui nous permet de nous concentrer directemement sur la reconnaissance des visages.







In [ ]:
# Récupération de tous les noms de fichiers de l'échantillon d'entraînement
file = readdir("Train")
trainFileName = ["Train/"*file[i] for i=1:length(file)];

In [ ]:
# Affichage des 8 photos de la première personne
load.(trainFileName[1:8])

In [ ]:
# Affichage des 8 photos de la huitième personne
load.(trainFileName[29:36])

In [ ]:
"""
    imgrayconvert(imageFileName ; columnStack=true ; T=Int64)

Conversion en intensité de gris de l'image du fichier `imageFileName`.

### Arguments
- `imageFileName::string` : le nom du fichier de l'image
- `columnStack::bool=true` : Si `true`, l'image est renvoyée comme un vecteur colonne (option par défaut) 
                             sinon la fonction renvoie la matrice des niveaux de gris.
- `T::DataType=Int64` : Type des éléments de la matrice (Int64 par défaut).

### Details
 
La fonction retourne la matrice ou le vecteur colonne des niveaux de gris.
 
### Examples

\```
 julia> imgrayconvert(imageFileName)
 julia> imgrayconvert(imageFileName ; columnStack=false)
 julia> imgrayconvert(imageFileName ; T=Float64)
\```

"""
function imgrayconvert(imageFileName::String ; columnStack::Bool=true)
    im = load(imageFileName)
    X = Float64.(im)
    if columnStack
        Y = X[:]
    else
        Y = X
    end
    return Y
end

In [ ]:
"""
    imshow(X), imshow(X, im_size)

Affiche une matrice ou un vecteur en une image composée de niveau de gris.

### Arguments
- `X::Array{Real}` : Une matrice ou un vecteur colonne à afficher.
- `im_size::Tuple{Int64,Int64}` : Un tuple de Int64 indicant la taille de l'image. 


### Details

L'argument `im_size` n'est nécessaire que si un vecteur colonne est envoyé comme image.

L'échelle des niveaux de gris est ajustée en fonction des valeurs contenues dans X.
 
### Examples

\```
 julia> imshow(X)
 julia> imshow(X, (m₁, m₂))
\```

"""
function imshow(X::Array{<:Real,1}, im_size::Tuple{Int64,Int64})
    
    # scale the eigenvector for display on grayscale
    m = minimum(X)
    M = maximum(X)
    
    Z = (X .- m) / (M-m)
    
    Z = reshape(Z, im_size)

    Gray.(Z)
    
end

function imshow(X::Array{<:Real,2})
    
    # scale the eigenvector for display on grayscale
    m = minimum(X)
    M = maximum(X)
    
    Z = (X .- m) / (M-m)
    
    Gray.(Z)
    
end

In [ ]:
# Chargement des images de l'ensemble d'entrainement. Chaque image correspond à une ligne de la matrice X, 
# chaque pixel à une colonne.

n = length(trainFileName)

im = imgrayconvert(trainFileName[1],columnStack=false)
m₁, m₂ = size(im)
m = m₁ * m₂

X = Array{Float64,2}(undef,n,m)

for i=1:n
   X[i,:] = imgrayconvert(trainFileName[i], columnStack=true) 
end


# 2. Analyse en composantes principales

Le but de cette section est de réduire la dimension du jeu de données d'entraînement. Nous ferons donc une décomposition en valeurs singulières de l'ensemble d'entraînement.

Les étapes sont les suivantes :
1. Centrer chacune des lignes de la matrice des visages d'entraînement pour obtenir la matrice $Z$.
2. Effectuer une décomposition en valeurs singulières de $Z$.
3. Choisir le nombre de composantes principales requises $k$. 

## 2.1 Centrer les images de l'ensemble d'entraînement

### a) Calculez le visage moyen $\bar{X}$ en faisait une moyenne de tous les visages pour chacun des pixels. Affichez le visage moyen avec la fonction `imshow`.

### b) Calculer la matrice $Z$ centrée des visages de l'ensemble d'entraînement. Ensuite, afficher la différence entre le premier visage  et le visage moyen avec la fonction `imshow`.

## 2.2 Décomposition en valeurs singulières de $Z$.

### a) Obtenez les matrices $U$ et $V$ ainsi que les valeurs singulières à l'aide de la fonction `svd` et/ou de la fonction `svdvals`.

### b) Affichez les premiers vecteurs singuliers de $V$ avec la fonction `imshow`. 

Ces composantes représentent les modes de plus grande variabilité. Dans la reconnaissance faciale, elles sont appelées les *eigenfaces*.

## 2.3 Choix du nombre $k$ de vecteurs singuliers à utiliser

### a) Tracez un graphique permettant de voir le pourcentage de la variance totale retenue en fonction du nombre de vecteur singuliers.

### b) Calculez le pourcentage de la variance récupérée en utilisant 10, 50 et 70 composantes principales.

# 3. Approximation d'une image à l'aide des vecteurs singuliers

Soit $k$ le nombre de vecteurs singuliers retenus. Nous allons approximer le visage $\mathbf{z}_i$ à l'aide des $k$ premières vecteurs singuliers. 

Dénotons par $V_k$ la matrice des $k$ premiers vecteurs singuliers de $V$. On cherche à trouver la combinaison linéaire des $k$ vecteurs qui approxime le visage $\mathbf{z}_i$ :

$$ \mathbf{Z}_i = V_k \mathbf{w} + \varepsilon ; $$

où $\mathbf{w}$ correspond au vecteur des poids de la combinaison linéaire et $\varepsilon$ correspond à l'erreur que l'on fait. L'erreur serait nulle si on utilisait tous les vecteurs singuliers.

On se retrouve donc dans un problème de régression où la variable d'intérêt est l'image du visage et où les variables explicatives sont les vecteur singuliers $V_k$. Pour trouver les coefficients de régression $\mathbf{w}$, il suffit de procéder comme au chapitre 2 :

$$ \mathbf{\hat{w}} = (V_k^\top V_k)^{-1} V_k^\top \mathbf{z}_i . $$

Or, puisque la matrice des vecteurs singuliers est orthonormales, la dernière équation se simplifie à l'expression suivante :

$$ \mathbf{\hat{w}} = V_k^\top \mathbf{z}_i . $$

Le visage $\mathbf{z}_i$ projeté dans les k premières composantes principales, dénoté par $\mathbf{\hat{z}}_i$,  est obtenue avec l'équation suivante :

$$ \mathbf{\hat{z}}_i = V_k \mathbf{\hat{w}}.$$



**Remarque :** On pourrait être de tenté de calculer la projection du visage en une seule étape, en remplaçant $\mathbf{w}$ par son expression :

$$ \mathbf{\hat{z}}_i = V_k \mathbf{\hat{w}} = V_k V_k^\top \mathbf{z}^\top_i.$$

La matrice $V_k V_k^\top$ correspond à une matrice de projection dans l'espace des $k$ premières composantes principales. Il faut cependant traiter le produit matricielle $V_k V_k^\top$ avec attention pour obtenir des résultats précis et rapides. Il est souvent plus judicieux de procéder en deux étapes :
1. Calculer les poids $\mathbf{\hat{w}}$.
2. Calculer la projection de l'image $\mathbf{\hat{z}}_i^\top = V_k \mathbf{\hat{w}}$.

## 3.1 Approximation du visage 1 avec les 10 premiers vecteurs singuliers

In [ ]:
i = 1
k = 10

Vk = V[:,1:k]
zᵢ = Z[i,:]

### a) Calculez les coefficients ŵ

### b) Calculez l'approximation du visage 

### c) Affichez le visage original et son approximation

Vous pouvez ajouter le visage moyen pour une meilleure interprétation

## 3.2 Approximation du visage 1 avec les 50 premières vecteurs singuliers

Reprenez les étapes précédentes mais cette fois en utilisant les 50 premiers vecteurs singuliers.

In [ ]:
i = 1
k = 50

Vk = V[:,1:k]
zᵢ = Z[i,:]



## 3.3 Approximation du visage 1 avec les 70 premières vecteurs singuliers

Reprenez les étapes précédentes mais cette fois en utilisant les 70 premiers vecteurs singuliers.

In [ ]:
i = 1
k = 70

Vk = V[:,1:k]
zᵢ = Z[i,:]


## 3.3 Approximation du visage 29 avec les 50 premières vecteurs singuliers

Reprenez les étapes précédentes mais cette fois avec le visage 29.

In [ ]:
i = 29
k = 50

Vk = V[:,1:k]
zᵢ = Z[i,:]



# 4. Reconnaissance faciale


L'idée de la reconnaissance faciale consiste à comparer le vecteur des coefficient $\mathbf{\hat{w}}$ de l'image à reconnaître avec les vecteurs poids des images de l'ensemble d'entraînement. C'est une comparaison assez facile à faire car ce vecteur est de dimension raisonnable comparativement aux images originales. En effet, dans notre cas, si on prend 50 vecteurs singuliers, le vecteur des coefficient est un vecteur colonne de taille 50. On peut donc résumer toutes les images par leur vecteur des coefficients de taille 50. 

Pour savoir, si une nouvelle image représente une personne présente dans l'ensemble d'entraînement, on n'a qu'à comparer son vecteur des coefficients avec chacun des vecteurs des coefficients de l'ensemble d'entraînement. Cette comparaison est facile, rapide et efficace car les vecteurs sont de tailles raisonnables. Si la différence entre les vecteurs est très grande, cela suggère que la personne est inconnue de l'ensemble d'entraînement. Si la différence est petite avec un des vecteurs de coefficients, cela suggère qu'il s'agit de la même personne. Le seuil doit être ajusté avec de la validation croisée.

Dans cette section, vous déciderez si une nouvelle image de l'ensemble test représente un personne connue ou inconnue. Vous le ferez en complétant les étapes suivantes :

1. Calculez les poids de toutes les images de l'ensemble d'entraînement.
2. Calculez les poids de l'image de test.
3. Calculez la distance entre les vecteurs poids des images d'entraînement et celui de l'image de test.
4. Identifiez l'image de l'ensemble d'entraînement la plus proche de l'image de test.
5. Décidez si le visage se retrouve dans l'échantillon d'entraînement ou s'il est inconnu.

### 4.1 Calculez les poids de toutes les images de l'ensemble d'entraînement

Prenez pour l'instant $k = 50$ composantes.

In [ ]:
k = 50
Vk = V[:,1:k]



### 4.2 Calculez les poids de l'image de test

Prenons d'abord la première image de l'échantillon de test. Il faut charger l'image, retirer le visage moyen et calculer les poids.


In [ ]:
# Récupération de tous les noms de fichiers de l'échantillon d'entraînement
file = readdir("Test")
testFileName = ["Test/"*file[i] for i=1:length(file)];

In [ ]:
# conversion de l'image de test en intensités de gris et retrait du visage moyen

j = 1  # j^e image de l'échantillon de test

y = imgrayconvert(testFileName[j]) - X̄

### 4.3 Calculez la distance euclidienne entre les vecteurs poids des images d'entraînement et celui de l'image de test.

### 4.4 Identifiez l'image de l'ensemble d'entraînement la plus proche de l'image de test.

### 4.5 Décidez si le visage se retrouve dans l'échantillon d'entraînement ou s'il est inconnu.

Si la distance minimale entre les poids des images d'entraînement et des poids de l'image de test, alors on statuera que le visage est inconnu. Il faut définir cependant définir ce seuil à l'aide de la validation croisée. Tentez d'utiliser le seuil de (3500)^2 et refaites les étapes du numéro 4 avec les autres images de l'ensemble de test.  

In [ ]:
# # Récupération des noms des personnes sur les images de l'ensemble d'entraînement
# ind1 = findfirst(isequal('/'),trainFileName[1])
# ind2 = findfirst(isequal('_'),trainFileName[1])
# trainPerson = [trainFileName[i][ind1+1:ind2-1] for i=1:length(trainFileName)];
